In [2]:
import random 
import json
import pickle 
import numpy as np 
import nltk
from nltk.stem import WordNetLemmatizer  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
import tkinter as tk
from tkinter import scrolledtext

In [3]:

# Assurez-vous d'avoir les ressources nécessaires de nltk
nltk.download('punkt')
nltk.download('wordnet')

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Charger les intents
with open('intents.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Traitement des données
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatization et suppression des signes de ponctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

# Sauvegarde des mots et des classes
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Préparer les données d'entraînement
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Vérifiez les tailles des vecteurs
print("Shape of bags and output rows:")
for item in training:
    print(len(item[0]), len(item[1]))

# Convertir en tableaux NumPy
train_x = np.array([x for x, _ in training])
train_y = np.array([y for _, y in training])


# Créer le modèle
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Créer un optimiseur SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compiler le modèle
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Sauvegarder le modèle
model.save('chatbot_model.h5',hist)
print('Done')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oussa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oussa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Shape of bags and output rows:
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305 42
305

In [4]:

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Charger les intents et le modèle
with open('intents.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result
    return "Sorry, I didn't get that."

print("Go! Bot is running!")
'''while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)'''


Go! Bot is running!


'while True:\n    message = input("")\n    ints = predict_class(message)\n    res = get_response(ints, intents)\n    print(res)'

In [5]:
# Création de l'application tkinter
def send():
    user_input = entry.get()
    if user_input:
        text_area.config(state=tk.NORMAL)
        text_area.insert(tk.END, "You: " + user_input + '\n')
        
        ints = predict_class(user_input)
        res = get_response(ints, intents)
        
        text_area.insert(tk.END, "Bot: " + res + '\n')
        text_area.config(state=tk.DISABLED)
        entry.delete(0, tk.END)

def exit_app():
    app.quit()

app = tk.Tk()
app.title("Chatbot")

# Zone de texte pour les messages
text_area = scrolledtext.ScrolledText(app, state=tk.DISABLED, wrap=tk.WORD)
text_area.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

# Champ de texte pour entrer un message
entry = tk.Entry(app, width=80)
entry.pack(pady=10, padx=10)

# Bouton d'envoi
send_button = tk.Button(app, text="Send", command=send)
send_button.pack(pady=10)

# Bouton de sortie
exit_button = tk.Button(app, text="Exit", command=exit_app)
exit_button.pack(pady=10)

app.mainloop()

1/1 [==============================] - 0s 31ms/step
